In [2]:
import sys
sys.path.append("../")


In [19]:
from sqlmodel import Session, select
from models.vault_performance import VaultPerformance
import pandas as pd
from core.db import engine

session = Session(engine)

query = """
SELECT datetime,total_locked_value
FROM public.vault_performance
WHERE vault_id = 'a09bd0e3-23b7-4529-adfd-09329b7a3012' and datetime > '2024-12-01' and datetime <'2024-12-03' 
"""

# Execute the query and load the results into a DataFrame
df = pd.read_sql(query, session.bind)
df["datetime"] = pd.to_datetime(df["datetime"])
if df["datetime"].dt.tz is None:
    df["datetime"] = df["datetime"].dt.tz_localize("UTC")
else:
    df["datetime"] = df["datetime"].dt.tz_convert("UTC")

df["datetime"] = df["datetime"].dt.floor("D")
df.set_index("datetime", inplace=True)

daily_df = df.resample("D").first()
daily_df.ffill(inplace=True)
daily_df.reset_index(inplace=True)
# Display the resulting DataFrame
daily_df

,datetime,total_locked_value
0,2024-12-01 00:00:00+00:00,1979.140352
1,2024-12-02 00:00:00+00:00,1998.131399


In [20]:
ALLOCATION_RATIO: float = 1 / 2
AE_USD = 0.08 / 365
LST_YEILD = 0.036 / 365

In [31]:
from datetime import datetime
from models.apy_rate_history import APYRateHistory
from models.funding_history import FundingHistory


ALLOCATION_RATIO: float = 1 / 2
AE_USD = 0.08 / 365
LST_YEILD = 0.036 / 365
RENZO_AEVO_VALUE: float = 0.065 / 365
SUPPORT_FEATURE_RATIO: float = 1 / 2
LEVERAGE: float = 4


def get_interest_rate_df(partner_name: str = "GOLDLINK"):
    query = select(APYRateHistory).where(APYRateHistory.partner_name == partner_name)
    data = session.exec(query).all()

    # Convert query results into a DataFrame
    df = pd.DataFrame(
        [{"datetime": item.datetime, "apy_rate": item.apy_rate} for item in data]
    )

    if df.empty:
        raise ValueError(f"No data found for partner: {partner_name}")

    # Convert 'datetime' column to UTC and normalize to the start of the day
    df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce").dt.normalize()

    # Resample to daily frequency, calculate mean funding rate, and forward-fill missing values
    interest_rate_avg = (
        df.set_index("datetime")
        .resample("D")["apy_rate"]
        # .mean()
        .ffill()  # Forward-fill missing data
        .reset_index()
    )

    # Rename columns to make them more descriptive
    interest_rate_avg.rename(
        columns={"datetime": "date", "apy_rate": "average_rate"},
        inplace=True,
    )

    return interest_rate_avg


def get_avg_by_date(avg_df: pd.DataFrame, target_date: datetime) -> float:
    avg_df["date_filter"] = avg_df["date"].dt.date

    filtered_row = avg_df.loc[avg_df["date_filter"] == target_date.date()]

    if not filtered_row.empty:
        print(filtered_row)
        return filtered_row["average_rate"].mean()
    return 0.0


def get_daily_funding_rate_df(partner_name: str):
    # Query FundingHistory data for the given partner_name
    query = select(FundingHistory).where(FundingHistory.partner_name == partner_name)
    data = session.exec(query).all()

    # Convert query results into a DataFrame
    df = pd.DataFrame(
        [
            {"datetime": item.datetime, "funding_rate": item.funding_rate}
            for item in data
        ]
    )

    if df.empty:
        raise ValueError(f"No data found for partner: {partner_name}")

    # Convert 'datetime' column to UTC and normalize to the start of the day
    df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")

    # Handle invalid or missing datetime values
    if df["datetime"].isna().any():
        raise ValueError("Invalid datetime format detected in the 'datetime' column")

    # Resample to daily frequency and calculate mean funding rate
    daily_avg = (
        df.set_index("datetime").resample("D")["funding_rate"].mean().reset_index()
    )

    # Rename column to make it more descriptive
    daily_avg.rename(
        columns={"datetime": "date", "funding_rate": "average_rate"},
        inplace=True,
    )
    return daily_avg


def get_tvl_from_prev_date(daily_df: pd.DataFrame, index: int) -> float:
    return float(daily_df.iloc[index]["total_locked_value"])

In [ ]:
PARTNER = {
    "BSX": "BSX",
    "AEVO": "AEVO",
    "HYPERLIQUID": "HYPERLIQUID",
    "GOLDLINK": "GOLDLINK",
}


def process_goldlink_vault():
    goldlink_funding_history_df = get_daily_funding_rate_df(PARTNER["GOLDLINK"])
    interest_rate_avg_df = get_interest_rate_df()

    for i, row in daily_df.iterrows():
        if i == 0:
            continue

        date = row["datetime"]
        prev_tvl = get_tvl_from_prev_date(daily_df, i - 1)
        prev_tvl = prev_tvl * LEVERAGE
        print("--------------------------------------------------------")
                print("prev_tvl",prev_tvl)
        print("--------------------------------------------------------")

        print("--------------------------------------------------------")
        print("funding_history_df")
        funding_history = get_avg_by_date(goldlink_funding_history_df, date)

        funding_history_avg = float(funding_history) / 365

        print("funding_history_avg", funding_history_avg)

        print("--------------------------------------------------------")
        print("interest_rate_df")
        interest_rate_avg = get_avg_by_date(interest_rate_avg_df, date)

        print("interest_rate_avg", interest_rate_avg)
        print("--------------------------------------------------------")
        # The funding rate of Goldlink is paid every 8 hours and is annualized

        # Calculate funding value based on the difference between the 1-day average funding rate
        # and the 1-day average interest rate, scaled by the TVL (Total Value Locked) and a multiplier of 4.
        # Formula: (Funding Rate Avg 1D - Interest Rate Avg 1D) * (TVL * 4)
        funding_value = (
            (funding_history_avg - interest_rate_avg) * ALLOCATION_RATIO * 24 * prev_tvl
        )

        print("funding_value", funding_value)
        yield_data = funding_value

In [40]:
process_goldlink_vault()

--------------------------------------------------------
funding_history_df
          date  average_rate date_filter
340 2024-12-02      0.510759  2024-12-02
funding_history_avg 0.0013993384773965318
--------------------------------------------------------
interest_rate_df
                          date  average_rate date_filter
1442 2024-12-02 00:00:00+00:00      0.127010  2024-12-02
1443 2024-12-02 00:00:00+00:00      0.127812  2024-12-02
1444 2024-12-02 00:00:00+00:00      0.127767  2024-12-02
1445 2024-12-02 00:00:00+00:00      0.127722  2024-12-02
1446 2024-12-02 00:00:00+00:00      0.127722  2024-12-02
1447 2024-12-02 00:00:00+00:00      0.127730  2024-12-02
1448 2024-12-02 00:00:00+00:00      0.127727  2024-12-02
1449 2024-12-02 00:00:00+00:00      0.127727  2024-12-02
1450 2024-12-02 00:00:00+00:00      0.129491  2024-12-02
1451 2024-12-02 00:00:00+00:00      0.129737  2024-12-02
1452 2024-12-02 00:00:00+00:00      0.129983  2024-12-02
1453 2024-12-02 00:00:00+00:00      0.1299